# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key


### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
weather_df = pd.read_csv('../WeatherPy/output_data/cities.csv')
weather_df.head()

,City,Latitude,Longitude,Max Temperature,Humidity,Cloudiness,Wind Speed,Country,Date
0,Ahipara,-35.1667,173.1667,67.98,82,68,4.90,NZ,1615609054
1,Bredasdorp,-34.5322,20.0403,51.80,93,45,1.14,ZA,1615609055
2,Cape Town,-33.9258,18.4232,61.00,94,0,10.36,ZA,1615609055
3,Busselton,-33.6500,115.3333,70.00,50,0,1.99,AU,1615609055
4,Qaanaaq,77.4840,-69.3632,-0.35,75,99,4.12,GL,1615609055


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
gmaps.configure(api_key=g_key)
#store coordinates
coordinates = weather_df[['Latitude','Longitude']]
weights = weather_df['Humidity'].astype(float)

# Plot Heatmap
fig = gmaps.figure()

# Create heat layer
heat_layer = gmaps.heatmap_layer(coordinates, weights=weights, 
                                 dissipating=False, max_intensity=10,
                                 point_radius=1)

# Add layer
fig.add_layer(heat_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [4]:
ideal_df = pd.DataFrame(weather_df, columns = ['City','Latitude','Longitude','Max Temperature', 'Wind Speed', 'Cloudiness','Country'])

ideal_temp = (ideal_df["Max Temperature"] < 80) & (ideal_df["Max Temperature"] > 70)
ideal_wind = ideal_df["Wind Speed"] < 10
ideal_clouds = ideal_df["Cloudiness"] == 0

hotel_df = ideal_df[ideal_temp & ideal_wind & ideal_clouds]

hotel_df

,City,Latitude,Longitude,Max Temperature,Wind Speed,Cloudiness,Country
115,Bentiu,9.2333,29.8333,77.92,2.37,0,SS
123,Huarmey,-10.0681,-78.1522,72.36,6.49,0,PE
129,East London,-33.0153,27.9116,75.20,8.05,0,ZA
172,Caravelas,-17.7125,-39.2481,72.36,4.88,0,BR
348,Oussouye,12.4850,-16.5469,73.40,2.30,0,SN
358,Beihai,21.4833,109.1000,75.45,6.35,0,CN
468,Cidreira,-30.1811,-50.2056,70.48,6.04,0,BR
562,Dali,25.7000,100.1833,70.95,1.72,0,CN


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [5]:
hotel_df['Hotel Name'] = ""
hotel_df

<ipython-input-5-cb2d5672db59>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hotel_df['Hotel Name'] = ""


,City,Latitude,Longitude,Max Temperature,Wind Speed,Cloudiness,Country,Hotel Name
115,Bentiu,9.2333,29.8333,77.92,2.37,0,SS,
123,Huarmey,-10.0681,-78.1522,72.36,6.49,0,PE,
129,East London,-33.0153,27.9116,75.20,8.05,0,ZA,
172,Caravelas,-17.7125,-39.2481,72.36,4.88,0,BR,
348,Oussouye,12.4850,-16.5469,73.40,2.30,0,SN,
358,Beihai,21.4833,109.1000,75.45,6.35,0,CN,
468,Cidreira,-30.1811,-50.2056,70.48,6.04,0,BR,
562,Dali,25.7000,100.1833,70.95,1.72,0,CN,


In [6]:
indices = []
indices = list(hotel_df.index.values)
indices

[115, 123, 129, 172, 348, 358, 468, 562]

In [7]:
hotels = []

for index in indices:
    lat = hotel_df.loc[index]['Latitude']
    lng = hotel_df.loc[index]['Longitude']

    params = {
        "location": f"{lat},{lng}",
        "radius": 5000,
        "types" : "hotel",
        "key": g_key
    }
    
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    response = requests.get(base_url, params=params)
    results = response.json()
    try:
        hotels.append(results['results'][0]['name'])
    except:
        hotels.append("")
        
hotel_df['Hotel Name'] = hotels
hotel_df = hotel_df.dropna(how='any')
hotel_df.head()

<ipython-input-7-74cf6eb38ba2>:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hotel_df['Hotel Name'] = hotels


,City,Latitude,Longitude,Max Temperature,Wind Speed,Cloudiness,Country,Hotel Name
115,Bentiu,9.2333,29.8333,77.92,2.37,0,SS,Bentiu
123,Huarmey,-10.0681,-78.1522,72.36,6.49,0,PE,Cercado de Huarmey
129,East London,-33.0153,27.9116,75.20,8.05,0,ZA,East London
172,Caravelas,-17.7125,-39.2481,72.36,4.88,0,BR,Caravelas
348,Oussouye,12.4850,-16.5469,73.40,2.30,0,SN,Oussouye


In [8]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[['Latitude', 'Longitude']]

In [9]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations)
fig.add_layer(markers)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))